In [ ]:
!pip install -q -U torch transformers langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

In [ ]:
!pip install -q -U scipy

In [ ]:
!playwright install
!playwright install-deps

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import torch
import transformers 
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
model_name="mistralai/Mistral-7B-Instruct-v0.1"
model_config=transformers.AutoConfig.from_pretrained(model_name, )
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

In [ ]:
compute_dtype=getattr(torch, "float16")
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)
major, _=torch.cuda.get_device_capability()
if major>=8:
    print("Yes it supports")
    

In [ ]:
if torch.cuda.is_available():
    num_devices=torch.cuda.device_count()
    for i in range(num_devices):
        print(f"CUDA Device{i}:{torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available")

In [ ]:
model=AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

In [ ]:
def no_parameters(model):
    all_model_params=0
    for _, param in model.named_parameters():
        all_model_params+=param.numel()
    return f"total number of parameters: {all_model_params}"

res=no_parameters(model)
print(f"{res}")

In [ ]:
text_generation_pipeline=pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=1000,
)
mistral_llm=HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
article= ["https://timesofindia.indiatimes.com/india/timestopten.cms"]
loader=AsyncChromiumLoader(article)
docs=loader.load()
html2text=Html2TextTransformer()
docs_transformed=html2text.transform_documents(docs)
text_splitter=CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
chunked_documents=text_splitter.split_documents(docs_transformed)

In [ ]:
db=FAISS.from_documents(chunked_documents, HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [ ]:
retriever=db.as_retriever()

In [ ]:
prompt_template="""
### [INST] Instruction: Check whether the given news is correct or not by referencing the original news given below:

{original_news}

### News
{News} [/INST]
"""
prompt=PromptTemplate(
    input_variables=["original_news", "News"], template=prompt_template,)


In [ ]:
llm_chain=LLMChain(llm=mistral_llm, prompt=prompt)
rag_chain=({"original_news":retriever, "News":RunnablePassthrough()} | llm_chain)


In [ ]:
result = rag_chain.invoke("Unfortunately Oppenheimer film couldn't get any awards at the Oscars this year")
output = f"News: {result['News']}\nText: {result['text']}"
print(output)